# 第11章 カルマンフィルタ

In [11]:
import numpy as np
from plot_filtering_test import * #実行フォルダに plot_filtering_test.py を置いてインポートします．

#### 乱数シードの指定（テキスト掲載用）
拙著に掲載の標本路を再現するため，シードを明示しておきます．本来は必要ないので，コメントアウトしてもいいです．

In [12]:
np.random.seed(123)

## クラスと関数

### 算法11.7 （定常カルマンフィルタ）

In [13]:
from class_ssKF import * #実行フォルダに class_ssKF.py を置いてインポートします． 

In [14]:
# 内容の確認
for line in open('./class_ssKF.py'): print(line[:-1])

import numpy as np
from scipy.linalg import solve_discrete_are

class class_ssKF: # Steady-state Kalman filter
    
    def __init__(s, F, G, H, Q, R, x0, cov0=None):
        
        ### システム行列
        s.F = np.array(F) #状態推移行列
        s.G = np.array(G) #駆動行列
        s.H = np.array(H) #観測行列
        # 雑音
        s.Q = np.array(Q) #システム雑音の共分散行列
        s.R = np.array(R) #観測雑音の共分散行列
        
        s.xdim = F.shape[1] #状態ベクトルの次元＝状態推移行列の列数
        s.ydim = H.shape[0] #観測行列の行数
        
        ### KBFの内部状態
        s.xf   = np.array(x0) #濾波推定値
        s.xp   = np.array(x0) #予測推定値
        s.cov0 = cov0         #共分散行列
        s.K    = None         #カルマンゲイン

        ### 定常カルマンフィルタの導出
        RicA = s.F.T
        RicB = s.H.T
        RicQ = s.G.dot(s.Q).dot(s.G.T)
        RicR = s.R
        #リカッチ方程式の解
        s.cov = solve_discrete_are(RicA, RicB, RicQ, RicR) 
        #定常カルマンゲイン
        HSH_R = s.H.dot(s.cov).dot(s.H.T)+s.R
        if HSH_R.ndim > 1:
            pinv = np.linalg.pinv(HSH_R)
  

## 11.5 数値例

In [ ]:
from class_LinearSDE import * #実行フォルダに class_LinearSDE.py を置いてインポートします． 

In [ ]:
# 内容の確認
#for line in open('./class_LinearSDE.py'): print(line[:-1])

In [ ]:
class Vib1DOF: # 連続時間1自由度振動系，フィルタは離散時間
    
    def __init__(self, c=0.3, k=1.0, Δt=0.1, Qval=1e-8, Rval=0.1 ):
        ### システムの定義
        # 状態行列
        self.A = np.array([[0,1],[-k,-c]])
        # 駆動行列
        self.D = np.array([[0],[1]]) #縦ベクトル 
        # 観測行列：1行2列と仮定
        self.C = np.array([[0,1]])
        # ノイズの条件
        self.Q = np.array([[Qval]]) #スカラは1x1行列として与える
        self.R = np.array([[Rval]]) #同上
        ### 出力データ
        self.tt = None   # 時刻の列
        self.xx = None   # 状態量の時系列
        self.yy = None   # 観測量の時系列
        self.xxf = None  # 濾波推定値の時系列
        self.xxni = None # 数値積分推定値の時系列
        ### その他
        self.Δt = Δt   # 数値積分の時間ステップ
        
    ### 標本路の取得
    def get_sample_path(s, x0=[1,0], tn=200):
        # 初期値とデータ長
        s.x0 = np.array(x0)
        s.tn = tn
        sde = class_LinearSDE(s.A, s.D, s.C, s.Q, s.R, s.x0, s.Δt)
        s.tt, s.xx, s.yy = sde.get_sample_path(tn)
        
    ### 濾波推定（離散時間定常カルマンフィルタ）
    def do_filtering(s):
        I = np.eye(len(s.x0))
        F = I + s.Δt*s.A
        G = np.sqrt(s.Δt)*s.D
        H = s.C
        x0 = s.x0
        cov0 = None
        kf = class_ssKF(F, G, H, s.Q, s.R, x0)
        s.xxf = np.zeros((s.tn+1,2))
        for t, yt in enumerate(s.yy):
            s.xxf[t,:] = kf.xf
            kf.filtering(yt)
        # フィルタの安定性の確認    
        kf.stability() 

    ### 数値積分による推定
    def do_NI_estimation(self):
        self.xxni = np.zeros((self.tn+1,2))
        xni = self.x0[0]
        for t, yt in enumerate(self.yy):
            self.xxni[t,0] = xni #観測された速度の数値積分値
            self.xxni[t,1] = yt  #観測された速度そのもの
            xni += yt*self.Δt

### 数値積分推定との比較

In [ ]:
# システム雑音 + 観測雑音
vib1 = Vib1DOF(Qval=0.01, Rval=0.05)
vib1.get_sample_path(tn=200)
vib1.do_filtering()
vib1.do_NI_estimation()
plot_filtering_test(vib1)
plt.savefig('figs/Ch11-Q1R1.eps', bbox_inches='tight')

In [ ]:
# システム雑音なし + 観測雑音
vib1 = Vib1DOF(Qval=0, Rval=0.05)
vib1.get_sample_path(tn=200)
vib1.do_filtering()
vib1.do_NI_estimation()
plot_filtering_test(vib1)

In [ ]:
# システム雑音 + 観測雑音なし
vib1 = Vib1DOF(Qval=0.01, Rval=0)
vib1.get_sample_path(tn=200)
vib1.do_filtering()
vib1.do_NI_estimation()
plot_filtering_test(vib1)